#Quantizing huggingface models to exl2
This version of my exl2 quantize colab creates a single quantizaion to upload privatly.\
To calculate an estimate for VRAM size use: [NyxKrage/LLM-Model-VRAM-Calculator](https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator)\
Not all models and architectures are compatible with exl2.\
I've only tested with llama-7b and mistral-7b, not sure if higher size models work with free colab.\
#Outdated
More recent stuff in [Anthonyg5005/hf-scripts](https://huggingface.co/Anthonyg5005/hf-scripts)\
If you need to quant a model to exl2 for free, check out the bot from the [Exllama Discord server](https://discord.gg/NSFwVuCjRq)

In [ ]:
#@title Download and install environment
!git clone https://github.com/turboderp/exllamav2
%cd exllamav2
print("Installing pip dependencies")
!pip install -q -r requirements.txt
!pip install -q huggingface_hub requests tqdm accelerate transformers
!pip install . -q
#@markdown Uses [download-model.py](https://github.com/oobabooga/text-generation-webui/blob/main/download-model.py) and [convert-to-safetensors.py](https://github.com/oobabooga/text-generation-webui/blob/main/convert-to-safetensors.py) by [oobabooga](https://github.com/oobabooga)
!wget https://raw.githubusercontent.com/oobabooga/text-generation-webui/main/download-model.py
!wget https://raw.githubusercontent.com/oobabooga/text-generation-webui/main/convert-to-safetensors.py
model = "none"
dsd = 'false'

In [ ]:
#@title Login to HF (Required to upload files)
#@markdown From my Colab/Kaggle login script on [Anthonyg5005/hf-scripts](https://huggingface.co/Anthonyg5005/hf-scripts/blob/main/HF%20Login%20Snippet%20Kaggle.py)
#import required functions
import os
from huggingface_hub import login, get_token, whoami

#get token
if os.environ.get('KAGGLE_KERNEL_RUN_TYPE', None) is not None: #check if user in kaggle
    from kaggle_secrets import UserSecretsClient
    from kaggle_web_client import BackendError
    try:
        login(UserSecretsClient().get_secret("HF_TOKEN")) #login if token secret found
    except BackendError:
        print('''
            When using Kaggle, make sure to use the secret key HF_TOKEN with a 'WRITE' token.
                   This will prevent the need to login every time you run the script.
                   Set your secrets with the secrets add-on on the top of the screen.
             ''')
if get_token() is not None:
    #if the token is found then log in:
    login(get_token())
else:
    #if the token is not found then prompt user to provide it:
    login(input("API token not detected. Enter your HuggingFace (WRITE) token: "))

#if the token is read only then prompt user to provide a write token (Only required if user needs a WRITE token, remove if READ is enough):
while True:
    if whoami().get('auth', {}).get('accessToken', {}).get('role', None) != 'write':
        if os.environ.get('HF_TOKEN', None) is not None: #if environ finds HF_TOKEN as read-only then display following text and exit:
            print('''
          You have the environment variable HF_TOKEN set.
          You cannot log in.
          Either set the environment variable to a 'WRITE' token or remove it.
                  ''')
            input("Press enter to continue.")
            exit()
        if os.environ.get('COLAB_BACKEND_VERSION', None) is not None:
            print('''
                              Your Colab secret key is read-only
                Please switch your key to 'write' or disable notebook access on the left.
                               For now, you are stuck in a loop
                  ''')
        elif os.environ.get('KAGGLE_KERNEL_RUN_TYPE', None) is not None:
            print('''
                                      Your Kaggle secret key is read-only
                Please switch your key to 'write' or unattach from notebook in add-ons at the top.
                          Having a read-only key attched will require login every time.
                ''')
        print("You do not have write access to this repository. Please use a valid token with (WRITE) access.")
        login(input("Enter your HuggingFace (WRITE) token: "))
        continue
    break

In [ ]:
#@title ##Choose HF model to download
#@markdown ###Repo should be formatted as user/repo
#@markdown Weights must be stored in safetensors
if model != "none":
    !rm -r {model}-exl2-{BPW}bpw
repo_url = "mistralai/Mistral-7B-Instruct-v0.2" # @param {type:"string"}
model = repo_url.replace("/", "_")
!python download-model.py {repo_url}
print("Model downloaded")
#@markdown Convert Pytorch weights to Safetensors
convert_safetensors = False # @param {type:"boolean"}
if convert_safetensors == True:
    !python convert-to-safetensors.py models/{model} --output models/{model}-st --bf16 --max-shard-size 1GB
    !rm -r models/{model}
    !mv models/{model}-st models/{model}
    print("Finished converting")
#@markdown If model files are stored in a pytorch .bin extention then enable convert_safetensors above.\
#@markdown ![Example Image](https://huggingface.co/Anthonyg5005/hf-scripts/resolve/main/ipynb/pytorch-example.jpg "File extension is .bin")

In [ ]:
#@title Quantize the model
#@markdown ###Quantization time will last based on model size
#@markdown Target bits per weight:
BPW = "4.125" # @param {type:"string"}
!mkdir {model}-exl2-{BPW}bpw-WD
!mkdir {model}-exl2-{BPW}bpw
!cp models/{model}/config.json {model}-exl2-{BPW}bpw-WD
#@markdown Calibrate with custom dataset, not recommended (optional):\
#@markdown **Do not mistake this with finetunining.** calibration is a different process that helps preserve model quality on smaller quants.
Calibrate = False # @param {type:"boolean"}
#@markdown Calibration dataset, enable calibrate above (must be filled out if calibrating):
if Calibrate == True:
    dataset_url = "https://huggingface.co/datasets/wikitext/resolve/refs%2Fconvert%2Fparquet/wikitext-103-v1/test/0000.parquet?download=true" # @param {type:"string"}
    dataset_url = dataset_url.replace("?download=true", "")
    if dsd == 'false':
        !wget {dataset_url}
        dsd = 'true'
    dataset = dataset_url.split("/")[-1]
    dataset_jtp = dataset.split(".")[0]
#@markdown To use a calibration dataset, enter the huggingface resolve url. Right click the download button and copy the link. Afterwards, paste the link into dataset_url. So far only supports public datasets.\
#@markdown ![Example Image](https://huggingface.co/Anthonyg5005/hf-scripts/resolve/main/ipynb/dataset-example.jpg "Copy from download button")\
#@markdown If dataset is jsonl then convert to parquet. (Not always reliable, must be formatted correctly)
convert_parquet = False # @param {type:"boolean"}
if convert_parquet == True:
    import pandas as pd #GitHub Copilot generated
    import pyarrow as pa #GitHub Copilot generated
    import pyarrow.parquet as pq #GitHub Copilot generated
    import json #GitHub Copilot generated

    def jsonl_to_parquet(jsonl_file, parquet_file): #GitHub Copilot generated
        # Read JSONL file line by line with explicit encoding
        with open(jsonl_file, 'r', encoding='utf-8') as f: #GitHub Copilot generated
            lines = f.readlines() #GitHub Copilot generated

        # Parse JSON lines and store as list of dictionaries
        data = [json.loads(line.strip()) for line in lines] #GitHub Copilot generated

        # Convert to Pandas DataFrame
        df = pd.DataFrame(data) #GitHub Copilot generated

        # Convert DataFrame to PyArrow Table
        table = pa.Table.from_pandas(df) #GitHub Copilot generated

        # Write PyArrow Table to Parquet file
        pq.write_table(table, parquet_file) #GitHub Copilot generated

    jsonl_to_parquet(dataset, f"{dataset_jtp}.parquet") #GitHub Copilot generated
    dataset = f"{dataset_jtp}.parquet"
#@markdown Quantizing only allows for parquet datasets to be used. Enable convert_parquet if your dataset ends in a .jsonl extention.\
#@markdown ![Example Image](https://huggingface.co/Anthonyg5005/hf-scripts/resolve/main/ipynb/jsonl-example.jpg "File extension is .jsonl")\
#@markdown pippa is used as an example in this image. This dataset contains content that is not suitable for users under 18. This dataset also may or may not be against colab TOS. It won't be allowed under free colab usage although you're able to use it with paid compute units.
if Calibrate == True:
    quant = f"convert.py -i models/{model} -o {model}-exl2-{BPW}bpw-WD -cf {model}-exl2-{BPW}bpw -c {dataset} -b {BPW}"
else:
    quant = f"convert.py -i models/{model} -o {model}-exl2-{BPW}bpw-WD -cf {model}-exl2-{BPW}bpw -b {BPW}"
!python {quant}

In [ ]:
#@title Upload to huggingface privately
#@markdown You may also set it to public but I'd recommend waiting for my next ipynb that will create mutliple quants and place them all into individual branches.
!rm -r {model}-exl2-{BPW}bpw-WD
!rm -r models/{model}
print("Uploading to Huggingface. May take a while")
from huggingface_hub import HfApi, whoami, create_repo
create_repo(f"{whoami().get('name', None)}/{model}-exl2-{BPW}bpw", private=True)
HfApi().upload_folder(folder_path=f"{model}-exl2-{BPW}bpw", repo_id=f"{whoami().get('name', None)}/{model}-exl2-{BPW}bpw", repo_type="model", commit_message="Upload from Colab automation")
print(f"uploaded to https://huggingface.co/{whoami().get('name', None)}/{model}-exl2-{BPW}bpw")